In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy.stats import skew

In [2]:
bmi = pd.read_csv('../CDC_tables/bmiagerev.csv')
weight = pd.read_csv('../CDC_tables/wtage.csv')
height = pd.read_csv('../CDC_tables/statage.csv')

In [3]:
for column in bmi.columns:
    bmi[column] = pd.to_numeric(bmi[column], errors='coerce')

In [4]:
def preprocess_who(df_dict_cdc):
    preprocessed_dict = {}
    for key, df_item in df_dict_cdc.items():
        df = df_item.copy()
        # Filter age
        df = df[(df['Agemos'] >= 7*12) & (df['Agemos'] <= 18*12)]
        df = df.rename(columns={'Agemos': 'Age'})
        preprocessed_dict[key] = df
    return preprocessed_dict

In [5]:
df_dict_cdc = {
    'hfa_b':height[height['Sex']==1],
    'hfa_g':height[height['Sex']==2],
    'wfa_b':weight[weight['Sex']==1],
    'wfa_g':weight[weight['Sex']==2],
    'bmifa_b':bmi[bmi['Sex']==1],
    'bmifa_g':bmi[bmi['Sex']==2]
}

In [6]:
processed_cdc = preprocess_who(df_dict_cdc)

In [7]:
df_1 = pd.read_spss('../final_data/caspian1 data.sav',convert_categoricals=True)
df_3 = pd.read_spss('../final_data/CASPIAN III.sav',convert_categoricals=True)
df_4 = pd.read_spss('../final_data/caspian4-ghorbani.sav',convert_categoricals=True)
df_5 = pd.read_spss('../final_data/caspian5-ghorbani.sav',convert_categoricals=True)
all_df = pd.read_csv('../final_data/merged_dataset.csv')

In [8]:
def rename_features(df1,caspian_number):
    df2 = df1.copy()
    if caspian_number==1:
        df2.rename(columns={'univer': 'university','district':'region', 'schoolty':'schoolType'}, inplace=True)    
    elif caspian_number==3:
        df2.rename(columns={'area':'region', 'heighte':'height','weighte':'weight'}, inplace=True)    
    elif caspian_number==4:
        df2.rename(columns={'weight_1': 'weight', 'height_2': 'height', 'universi': 'university','waist_3':'waist','hip_4':'hip','wrist_5':'wrist'}, inplace=True)        
    elif caspian_number==5:
        df2.rename(columns={'weight_1': 'weight', 'height_2': 'height', 'universi': 'university','ap_9':'schoolType','waist_3':'waist','wrist4':'wrist'}, inplace=True)


    return df2

In [9]:
df_1=rename_features(df_1,1)
df_3=rename_features(df_3,3)
df_4=rename_features(df_4,4)
df_5=rename_features(df_5,5)
#add hip column to caspain 3
df_3['hip'] = np.nan

In [10]:
df_1['sex'] = df_1['sex'].apply(lambda x: 'Girl' if x == 'Female' else 'Boy' if x == 'Male' else x)
df_3['sex'] = df_3['sex'].apply(lambda x: 'Girl' if x == 'male' else 'Boy' if x == 'female' else x)
df_4['sex'] = df_4['sex'].apply(lambda x: 'Girl' if x == 'girl' else 'Boy' if x == 'boy' else x)
df_5['sex'] = df_5['sex'].apply(lambda x: 'Girl' if x == 'girl' else 'Boy' if x == 'boy' else x)



In [11]:
def preprocess(dataframes_dict):
    processed_dfs = {}  # Dictionary to store processed DataFrames
    for name, df_org in dataframes_dict.items():
        df = df_org.copy()

        # Filter age
        df = df[(df["age"] >= 7) & (df["age"] <= 18)]
        # please change the type of heught_1 and weight_1 in caspian4 to numeric if you can't do it directly uncomment two line below
        df['height'] = pd.to_numeric(df['height'], errors='coerce')
        df['weight'] = pd.to_numeric(df['weight'], errors='coerce')
        df["bmi1"] = df["weight"] / ((df["height"] / 100) ** 2)
        
        # Remove null tuples
        records_with_nulls = df[
            df[["sex"]].isna().any(axis=1)
        ]
        df = df.dropna(subset=["sex"])
        print(
            f"Number of records with NaN value in sex in {name}: {len(records_with_nulls)}"
        )

        # Store the processed DataFrame in the new dictionary
        processed_dfs[name] = df

    return processed_dfs

df_dict = {'casp1' :df_1 ,'casp3': df_3, 'casp4': df_4,'casp5':df_5, 'All-Dfs':all_df}
processed_dfs = preprocess(df_dict)


Number of records with NaN value in sex in casp1: 1
Number of records with NaN value in sex in casp3: 0
Number of records with NaN value in sex in casp4: 0
Number of records with NaN value in sex in casp5: 0
Number of records with NaN value in sex in All-Dfs: 0


In [12]:
def calculate_z_score(value, L, M, S):
    """
    Calculate the z-score using the LMS method.

    Parameters:
    - value: Observed value (e.g., height, weight, BMI).
    - L: Lambda (skewness parameter).
    - M: Mu (median or central tendency).
    - S: Sigma (coefficient of variation).

    Returns:
    - z_score: Standardized z-score.
    """
    if L == 0:
        z_score = (value / M - 1) / S
    else:
        z_score = (np.power(value / M, L) - 1) / (L * S)
    return z_score

def apply_z_score(group, params,feature):
    # Merge params (Lambda, Median, Sigma) with the group
    group = group.merge(params, on='sex', how='left')
    
    # Apply the z-score calculation row-wise within each group
    group['Z-Score'] = group.apply(
        lambda row: calculate_z_score(row[f'{feature}'], L=row['Lambda'], M=row['Median'], S=row['Sigma']), axis=1
    )
    return group

def calculate_params(group, feature):
    # Calculate Median (M)
    M = group[f'{feature}'].median()

    # Calculate Lambda (L) - Skewness
    L = skew(group[f'{feature}'])

    # Calculate Sigma (S) - Coefficient of Variation (std / mean)
    mean = group[f'{feature}'].mean()
    std_dev = group[f'{feature}'].std()
    S = std_dev / mean if mean != 0 else None  # Avoid division by zero

    return pd.Series({'Lambda': L, 'Median': M, 'Sigma': S})

In [13]:
feature_sex_to_key = {
    ('height', 'Boy'): 'hfa_b',
    ('height', 'Girl'): 'hfa_g',
    ('weight', 'Boy'): 'wfa_b',
    ('weight', 'Girl'): 'wfa_g',
    ('bmi1', 'Boy'): 'bmifa_b',
    ('bmi1', 'Girl'): 'bmifa_g',
}
def get_processed_cdc_dataset(feature, sex):
    key = feature_sex_to_key.get((feature, sex))
    if key is None:
        raise ValueError(f"No dataset found for feature: {feature} and sex: {sex}")
    return processed_cdc[key]

In [16]:
for name_df, df_org in processed_dfs.items():
    grouped = df_org.groupby(['age', 'sex'])

    percentile_data = []
    percentile = [3,5,10,25,50,75,90,95,97]
    percentiles_labels = ['P3','P5', 'P10','P25', 'P50', 'P75', 'P90', 'P95','P97']
    # Now, apply this to each group, passing the params DataFrame
    frac = 0.7
    features = ['height','weight']
    colors = plt.cm.tab10(np.linspace(0, 1, len(percentile)))  # Using the 'plasma' colormap

# Loop through each feature to calculate percentiles
    for feature in features:
        params = grouped.apply(lambda group: calculate_params(group, feature=feature)).reset_index()
        grouped_with_z_score = grouped.apply(lambda group: apply_z_score(group, params, feature))
        ages = sorted(grouped_with_z_score['age_x'].unique())
        sexes = grouped_with_z_score['sex'].unique()
        
        for sex in sexes:
            percentile_data = []  # Clear data for each sex
            baseline_data = get_processed_cdc_dataset(feature, sex)  # Fetch dynamically
            ages_who = baseline_data['Age'] / 12
            # common_ages = np.intersect1d(ages, ages_who.unique())
            # print(common_ages)
            for age in ages:
                feature_data = grouped_with_z_score[
                    (grouped_with_z_score['age_x'] == age) & (grouped_with_z_score['sex'] == sex)
                ][feature]

                # Handle missing or invalid data
                feature_data = pd.to_numeric(feature_data, errors='coerce').dropna()
                if feature_data.empty:
                    print(f"No valid data for sex: {sex}, age: {age}. Skipping...")
                    continue

                # Calculate percentiles
                percentiles_values = np.percentile(feature_data, percentile)
                percentile_data.append([sex, age] + percentiles_values.tolist())

            if not percentile_data:
                print(f"No data for {sex} in feature {feature}. Skipping...")
                continue

            # Create DataFrame from collected data
            percentile_columns = ['Gender', 'Age'] + percentiles_labels
            percentile_df = pd.DataFrame(percentile_data, columns=percentile_columns)

            # Plotting
            fig, ax = plt.subplots(figsize=(12, 6))
            sex_data = percentile_df[percentile_df['Gender'] == sex]

            # Smoothed custom percentiles
            for idx, percentile_label in enumerate(percentiles_labels):
                smoothed_percentile = lowess(sex_data[percentile_label], sex_data['Age'], frac=frac)
                ax.plot(
                    smoothed_percentile[:, 0],  # Ages
                    smoothed_percentile[:, 1],  # Smoothed values
                    linestyle='-',
                    label=f'{percentile_label} Percentile',color=colors[idx]
                )

            # Add CDC baseline percentiles
            for idx, percentile_label in enumerate(percentiles_labels):
                ax.plot(
                    ages_who,  # Ages
                    baseline_data[percentile_label],  # CDC baseline values
                    linestyle='-.',

                    label=f'{percentile_label} Percentile CDC',color=colors[idx]
                )


            # Title, labels, and legend
            ax.set_title(f'{name_df}-{feature}-Percentiles by Age-{sex.capitalize()}-LMS Method-Compare to CDC')
            ax.set_xlabel('Age')
            ax.set_ylabel(feature.capitalize())
            ax.legend()
            ax.grid()
            ax.set_xticks(sex_data['Age'])

            # Save and show
            plt.savefig(f'../compare_CDC_chart1/{name_df}-{feature}-{sex.capitalize()}.png')
            # plt.show()
            plt.close()
            percentile_data.clear()
            print('sure',len(percentile_data))



/tmp/ipykernel_20846/4222155261.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df_org.groupby(['age', 'sex'])
/tmp/ipykernel_20846/4222155261.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  params = grouped.apply(lambda group: calculate_params(group, feature=feature)).reset_index()
/tmp/ipykernel_20846/4222155261.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be ex

sure 0
sure 0


/tmp/ipykernel_20846/4222155261.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  params = grouped.apply(lambda group: calculate_params(group, feature=feature)).reset_index()
/tmp/ipykernel_20846/4222155261.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_with_z_score = grouped.apply(lambda group: apply_z_score(group, params, feature))


sure 0
sure 0


/tmp/ipykernel_20846/4222155261.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df_org.groupby(['age', 'sex'])
/tmp/ipykernel_20846/4222155261.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  params = grouped.apply(lambda group: calculate_params(group, feature=feature)).reset_index()
/tmp/ipykernel_20846/4222155261.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be ex

...

sure 0
sure 0


/tmp/ipykernel_20846/4222155261.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  params = grouped.apply(lambda group: calculate_params(group, feature=feature)).reset_index()
/tmp/ipykernel_20846/4222155261.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_with_z_score = grouped.apply(lambda group: apply_z_score(group, params, feature))


sure 0
sure 0


/tmp/ipykernel_20846/4222155261.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  params = grouped.apply(lambda group: calculate_params(group, feature=feature)).reset_index()
/tmp/ipykernel_20846/4222155261.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_with_z_score = grouped.apply(lambda group: apply_z_score(group, params, feature))


sure 0
sure 0


In [17]:
for name_df, df_org in processed_dfs.items():
    grouped = df_org.groupby(['age', 'sex'])

    percentile_data = []
    percentile = [3,5,10,25,50,75,85,90,95,97]
    percentiles_labels = ['P3','P5', 'P10','P25', 'P50', 'P75','P85', 'P90', 'P95','P97']
    # Now, apply this to each group, passing the params DataFrame
    frac = 0.7
    features = ['bmi1']
    colors = plt.cm.tab10(np.linspace(0, 1, len(percentile)))  # Using the 'plasma' colormap

# Loop through each feature to calculate percentiles
    for feature in features:
        params = grouped.apply(lambda group: calculate_params(group, feature=feature)).reset_index()
        grouped_with_z_score = grouped.apply(lambda group: apply_z_score(group, params, feature))
        ages = sorted(grouped_with_z_score['age_x'].unique())
        sexes = grouped_with_z_score['sex'].unique()
        
        for sex in sexes:
            percentile_data = []  # Clear data for each sex
            baseline_data = get_processed_cdc_dataset(feature, sex)  # Fetch dynamically
            print(baseline_data)
            ages_who = baseline_data['Age'] / 12
            # common_ages = np.intersect1d(ages, ages_who.unique())
            # print(common_ages)
            for age in ages:
                feature_data = grouped_with_z_score[
                    (grouped_with_z_score['age_x'] == age) & (grouped_with_z_score['sex'] == sex)
                ][feature]

                # Handle missing or invalid data
                feature_data = pd.to_numeric(feature_data, errors='coerce').dropna()
                if feature_data.empty:
                    print(f"No valid data for sex: {sex}, age: {age}. Skipping...")
                    continue

                # Calculate percentiles
                percentiles_values = np.percentile(feature_data, percentile)
                percentile_data.append([sex, age] + percentiles_values.tolist())

            if not percentile_data:
                print(f"No data for {sex} in feature {feature}. Skipping...")
                continue

            # Create DataFrame from collected data
            percentile_columns = ['Gender', 'Age'] + percentiles_labels
            percentile_df = pd.DataFrame(percentile_data, columns=percentile_columns)

            # Plotting
            fig, ax = plt.subplots(figsize=(12, 6))
            sex_data = percentile_df[percentile_df['Gender'] == sex]

            # Smoothed custom percentiles
            for idx, percentile_label in enumerate(percentiles_labels):
                smoothed_percentile = lowess(sex_data[percentile_label], sex_data['Age'], frac=frac)
                ax.plot(
                    smoothed_percentile[:, 0],  # Ages
                    smoothed_percentile[:, 1],  # Smoothed values
                    linestyle='-',
                    label=f'{percentile_label} Percentile',color=colors[idx]
                )

            # Add CDC baseline percentiles
            for idx, percentile_label in enumerate(percentiles_labels):
                print(baseline_data[percentile_label])
                ax.plot(
                    ages_who,  # Ages
                    baseline_data[percentile_label],  # CDC baseline values
                    linestyle='-.',
                    label=f'{percentile_label} Percentile CDC',color=colors[idx]
                )


            # Title, labels, and legend
            ax.set_title(f'{name_df}-{feature}-Percentiles by Age-{sex.capitalize()}-LMS Method-Compare to CDC')
            ax.set_xlabel('Age')
            ax.set_ylabel(feature.capitalize())
            ax.legend()
            ax.grid()
            ax.set_xticks(sex_data['Age'])

            # Save and show
            plt.savefig(f'../compare_CDC_chart1/{name_df}-{feature}-{sex.capitalize()}.png')
            # plt.show()
            plt.close()
            percentile_data.clear()
            print('sure',len(percentile_data))



/tmp/ipykernel_20846/3977857459.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df_org.groupby(['age', 'sex'])
/tmp/ipykernel_20846/3977857459.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  params = grouped.apply(lambda group: calculate_params(group, feature=feature)).reset_index()
/tmp/ipykernel_20846/3977857459.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be ex

     Sex    Age         L          M         S         P3         P5  \
281  2.0   84.5 -2.926187  15.453565  0.105325  13.218157  13.432760   
282  2.0   85.5 -2.899435  15.479910  0.106320  13.221251  13.437931   
283  2.0   86.5 -2.872731  15.507184  0.107316  13.225019  13.443803   
284  2.0   87.5 -2.846124  15.535368  0.108313  13.229462  13.450374   
285  2.0   88.5 -2.819658  15.564444  0.109308  13.234580  13.457640   
..   ...    ...       ...        ...       ...        ...        ...   
408  2.0  211.5 -2.269732  21.132158  0.147287  17.045301  17.422184   
409  2.0  212.5 -2.276917  21.162062  0.147260  17.072360  17.449352   
410  2.0  213.5 -2.283925  21.191335  0.147245  17.098644  17.475755   
411  2.0  214.5 -2.290731  21.219975  0.147241  17.124126  17.501370   
412  2.0  215.5 -2.297324  21.247973  0.147248  17.148786  17.526177   

           P10        P25        P50        P75        P85        P90  \
281  13.791903  14.487649  15.453565  16.734618  17.625567  18

/tmp/ipykernel_20846/3977857459.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df_org.groupby(['age', 'sex'])
/tmp/ipykernel_20846/3977857459.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  params = grouped.apply(lambda group: calculate_params(group, feature=feature)).reset_index()
/tmp/ipykernel_20846/3977857459.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be ex

     Sex    Age         L          M         S         P3         P5  \
61   1.0   84.5 -3.323189  15.512869  0.092131  13.528743  13.721129   
62   1.0   85.5 -3.317827  15.530336  0.092946  13.530246  13.723873   
63   1.0   86.5 -3.310924  15.548758  0.093765  13.532330  13.727243   
64   1.0   87.5 -3.302612  15.568121  0.094589  13.535001  13.731241   
65   1.0   88.5 -3.293018  15.588411  0.095417  13.538263  13.735870   
..   ...    ...       ...        ...       ...        ...        ...   
188  1.0  211.5 -1.902639  21.628539  0.132521  17.637337  18.023329   
189  1.0  212.5 -1.896630  21.682621  0.132462  17.680816  18.068019   
190  1.0  213.5 -1.890816  21.736404  0.132409  17.723986  18.112395   
191  1.0  214.5 -1.885210  21.789880  0.132361  17.766833  18.156440   
192  1.0  215.5 -1.879824  21.843038  0.132320  17.809343  18.200143   

           P10        P25        P50        P75        P85        P90  \
61   14.042159  14.660824  15.512869  16.631117  17.401219  18

/tmp/ipykernel_20846/3977857459.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  params = grouped.apply(lambda group: calculate_params(group, feature=feature)).reset_index()
/tmp/ipykernel_20846/3977857459.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_with_z_score = grouped.apply(lambda group: apply_z_score(group, params, feature))


     Sex    Age         L          M         S         P3         P5  \
61   1.0   84.5 -3.323189  15.512869  0.092131  13.528743  13.721129   
62   1.0   85.5 -3.317827  15.530336  0.092946  13.530246  13.723873   
63   1.0   86.5 -3.310924  15.548758  0.093765  13.532330  13.727243   
64   1.0   87.5 -3.302612  15.568121  0.094589  13.535001  13.731241   
65   1.0   88.5 -3.293018  15.588411  0.095417  13.538263  13.735870   
..   ...    ...       ...        ...       ...        ...        ...   
188  1.0  211.5 -1.902639  21.628539  0.132521  17.637337  18.023329   
189  1.0  212.5 -1.896630  21.682621  0.132462  17.680816  18.068019   
190  1.0  213.5 -1.890816  21.736404  0.132409  17.723986  18.112395   
191  1.0  214.5 -1.885210  21.789880  0.132361  17.766833  18.156440   
192  1.0  215.5 -1.879824  21.843038  0.132320  17.809343  18.200143   

           P10        P25        P50        P75        P85        P90  \
61   14.042159  14.660824  15.512869  16.631117  17.401219  18

/tmp/ipykernel_20846/3977857459.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  params = grouped.apply(lambda group: calculate_params(group, feature=feature)).reset_index()
/tmp/ipykernel_20846/3977857459.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_with_z_score = grouped.apply(lambda group: apply_z_score(group, params, feature))


     Sex    Age         L          M         S         P3         P5  \
61   1.0   84.5 -3.323189  15.512869  0.092131  13.528743  13.721129   
62   1.0   85.5 -3.317827  15.530336  0.092946  13.530246  13.723873   
63   1.0   86.5 -3.310924  15.548758  0.093765  13.532330  13.727243   
64   1.0   87.5 -3.302612  15.568121  0.094589  13.535001  13.731241   
65   1.0   88.5 -3.293018  15.588411  0.095417  13.538263  13.735870   
..   ...    ...       ...        ...       ...        ...        ...   
188  1.0  211.5 -1.902639  21.628539  0.132521  17.637337  18.023329   
189  1.0  212.5 -1.896630  21.682621  0.132462  17.680816  18.068019   
190  1.0  213.5 -1.890816  21.736404  0.132409  17.723986  18.112395   
191  1.0  214.5 -1.885210  21.789880  0.132361  17.766833  18.156440   
192  1.0  215.5 -1.879824  21.843038  0.132320  17.809343  18.200143   

           P10        P25        P50        P75        P85        P90  \
61   14.042159  14.660824  15.512869  16.631117  17.401219  18

/tmp/ipykernel_20846/3977857459.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  params = grouped.apply(lambda group: calculate_params(group, feature=feature)).reset_index()
/tmp/ipykernel_20846/3977857459.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_with_z_score = grouped.apply(lambda group: apply_z_score(group, params, feature))


     Sex    Age         L          M         S         P3         P5  \
61   1.0   84.5 -3.323189  15.512869  0.092131  13.528743  13.721129   
62   1.0   85.5 -3.317827  15.530336  0.092946  13.530246  13.723873   
63   1.0   86.5 -3.310924  15.548758  0.093765  13.532330  13.727243   
64   1.0   87.5 -3.302612  15.568121  0.094589  13.535001  13.731241   
65   1.0   88.5 -3.293018  15.588411  0.095417  13.538263  13.735870   
..   ...    ...       ...        ...       ...        ...        ...   
188  1.0  211.5 -1.902639  21.628539  0.132521  17.637337  18.023329   
189  1.0  212.5 -1.896630  21.682621  0.132462  17.680816  18.068019   
190  1.0  213.5 -1.890816  21.736404  0.132409  17.723986  18.112395   
191  1.0  214.5 -1.885210  21.789880  0.132361  17.766833  18.156440   
192  1.0  215.5 -1.879824  21.843038  0.132320  17.809343  18.200143   

           P10        P25        P50        P75        P85        P90  \
61   14.042159  14.660824  15.512869  16.631117  17.401219  18